# Extract Document Sections from PAD

Identify and extract major document sections from PAD markdown files using OpenAI API.

**Note:** The reusable extraction logic has been implemented in `src/extraction/`. Use the CLIs for production:
- Extract sections: `uv run python -m src.extraction.cli_sections`
- Extract abbreviations: `uv run python -m src.extraction.cli_abbreviations`
- Create chunked markdown files: `uv run python -m src.extraction.cli_chunks`

This notebook contains the original exploration and can be used for testing individual files.

## 0. Setup

### 0.01 Import Required Libraries

In [2]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

from openai import OpenAI

# Import our config
import sys
sys.path.append(str(Path.cwd().parent))
from src.config import load_config

### 0.02 Load Configuration and Environment Variables

In [3]:
# Load environment variables from .env file
project_root = Path.cwd().parent
env_path = project_root / ".env"

if not env_path.exists():
    raise FileNotFoundError(
        f"'.env' file not found at {env_path}\n"
        "Please copy .env.example to .env and add your OpenAI API key."
    )

# Load from specific path
load_dotenv(env_path, override=True)

# Load project config
config = load_config()

# Get OpenAI API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Verify API key is set
if not OPENAI_API_KEY:
    raise ValueError("Missing required environment variable: OPENAI_API_KEY")

print("✓ Environment variables loaded")
print(f"  API Key: {OPENAI_API_KEY[:10]}...{OPENAI_API_KEY[-4:]}")

✓ Environment variables loaded
  API Key: sk-proj-cj...__0A


### 0.03 Set Up Paths

In [4]:
# Get paths
md_dir = project_root / config.paths.markdown
prompts_dir = project_root / "prompts"

# Target markdown file
target_file = md_dir / "P075941_1.md"
project_id = "P075941"

print(f"Markdown directory: {md_dir}")
print(f"Prompts directory: {prompts_dir}")
print(f"Target file: {target_file}")
print(f"File exists: {target_file.exists()}")

Markdown directory: /Users/lauren/repos/PAD2Skills/data/silver/pads_md
Prompts directory: /Users/lauren/repos/PAD2Skills/prompts
Target file: /Users/lauren/repos/PAD2Skills/data/silver/pads_md/P075941_1.md
File exists: True


## 1. Load Markdown Content

In [5]:
# Load markdown file
if not target_file.exists():
    raise FileNotFoundError(f"Markdown file not found: {target_file}")

markdown_content = target_file.read_text(encoding="utf-8")

# Prepare input message for custom GPT
input_message = f"Project ID: {project_id}\n\n{markdown_content}"

print(f"✓ Markdown content loaded ({len(markdown_content)} chars)")
print(f"Input message length: {len(input_message)} chars")
print(f"\nMarkdown preview (first 500 chars):")
print("=" * 60)
print(markdown_content[:500])
print("...")

✓ Markdown content loaded (736923 chars)
Input message length: 736944 chars

Markdown preview (first 500 chars):
Public Disclosure Authorized

Public Disclosure Authorized

Public Disclosure Authorized Public Disclosure Authorized

Public Disclosure Authorized

Public Disclosure Authorized

Document of The World Bank

## FOR OFFICIAL USE ONLY

## INTERNATIONAL DEVELOPMENT ASSOCIATION

## PROJECT APPRAISAL DOCUMENT

ON  A PROPOSED CREDIT IN THE AMOUNT OF  SDR 37.80  MILLION (US$56.65 MILLION EQUIVALENT) AND A PROPOSED GRANT IN THE AMOUNT OF SDR 37.80 MILLION (US$ 56.65 MILLION EQUIVALENT) TO  THE  REPUBLIC 
...


## 2. Initialize OpenAI Client

In [6]:
# Initialize OpenAI client
client = OpenAI()

print("✓ OpenAI client initialized")

✓ OpenAI client initialized


## 3. Send Request to Identify Document Sections

In [ ]:
print("Sending request to custom GPT...")
print(f"Input length: {len(input_message)} chars")
print()

# Call custom GPT with prompt ID
response = client.responses.create(
    prompt={
        "id": "pmpt_6950b4992fcc8194b89fc2d87be08bf8088afcd3c3f3a4d7",
        "version": "6"
    },
    input=[
        {"role": "user", "content": input_message}
    ],
    reasoning={
        "summary": None
    },
    store=True,
    include=[
        "reasoning.encrypted_content",
        "web_search_call.action.sources"
    ]
)

Sending request to custom GPT...
Input length: 736944 chars



In [8]:
# Extract the text from the response
# response.output is a list: [ResponseReasoningItem, ResponseOutputMessage]
# The message content is in output[1].content[0].text
for item in response.output:
    if hasattr(item, 'content') and hasattr(item, 'role'):
        result = item.content[0].text
        break

print("✓ Response received")
print(f"  Response ID: {response.id}")
print(f"  Status: {response.status}")


✓ Response received
  Response ID: resp_07309278ff531aad0069573f3d960881919e805586cf991eb1
  Status: completed


## 4. Save Results to File

In [9]:
# Save result directly to JSON file
output_dir = project_root / "data" / "silver" / "document_sections"
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / f"{project_id}_sections.json"

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(result)

print(f"✓ Saved sections to: {output_file}")
print(f"  File size: {output_file.stat().st_size / 1024:.2f} KB")


✓ Saved sections to: /Users/lauren/repos/PAD2Skills/data/silver/document_sections/P075941_sections.json
  File size: 2.73 KB


## 5. Read and Parse Section JSON

In [12]:
# Read and parse the sections JSON file
with open(output_file, 'r', encoding='utf-8') as f:
    sections_data = json.load(f)

sections = sections_data['sections']

print(f"✓ Loaded {len(sections)} sections")
for section in sections[:5]:  # Show first 5
    print(f"  {section['section_id']}: {section['section_title']}")
if len(sections) > 5:
    print(f"  ... and {len(sections) - 5} more")


✓ Loaded 16 sections
  0: STRATEGIC CONTEXT
  1: PROJECT DEVELOPMENT OBJECTIVES
  2: PROJECT DESCRIPTION
  3: IMPLEMENTATION
  4: KEY RISKS AND MITIGATION MEASURES
  ... and 11 more


## 6. Split Markdown into Section Chunks

In [13]:
import re

def to_snake_case(text):
    """Convert text to lower snake case"""
    # Replace spaces and special chars with underscores
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', '_', text)
    return text.lower()

def normalize_whitespace(text):
    """Normalize multiple spaces to single space"""
    return re.sub(r'\s+', ' ', text)

def find_header_in_markdown(markdown, header_text):
    """Find header in markdown, handling whitespace differences"""
    # Try exact match first
    pos = markdown.find(header_text)
    if pos != -1:
        return pos
    
    # Try with normalized whitespace
    normalized_header = normalize_whitespace(header_text)
    
    # Search using regex to match any amount of whitespace
    pattern = re.escape(normalized_header).replace(r'\ ', r'\s+')
    match = re.search(pattern, markdown)
    
    if match:
        return match.start()
    
    return -1

# Create output directory for chunks
chunks_dir = md_dir.parent / "pads_md_chunks"
chunks_dir.mkdir(parents=True, exist_ok=True)

print(f"Splitting markdown into {len(sections)} section chunks...")
print(f"Output directory: {chunks_dir}")
print()

# Split the markdown by sections
saved_chunks = []
for i, section in enumerate(sections):
    header_text = section['header_text']
    section_id = section['section_id']
    section_title = section['section_title']
    
    # Find the start position of this section (with fuzzy whitespace matching)
    start_pos = find_header_in_markdown(markdown_content, header_text)
    
    if start_pos == -1:
        print(f"⚠ Warning: Could not find header '{header_text}' in markdown")
        continue
    
    # Find the end position (start of next section, or end of document)
    if i < len(sections) - 1:
        next_header = sections[i + 1]['header_text']
        end_pos = find_header_in_markdown(markdown_content[start_pos + len(header_text):], next_header)
        if end_pos == -1:
            end_pos = len(markdown_content)
        else:
            end_pos = start_pos + len(header_text) + end_pos
    else:
        end_pos = len(markdown_content)
    
    # Extract the section content
    section_content = markdown_content[start_pos:end_pos].rstrip()
    
    # Generate filename
    snake_title = to_snake_case(section_title)
    filename = f"{project_id}_{section_id}_{snake_title}.md"
    chunk_file = chunks_dir / filename
    
    # Save the chunk
    chunk_file.write_text(section_content, encoding='utf-8')
    saved_chunks.append({
        'file': filename,
        'size': len(section_content),
        'title': section_title
    })
    
    print(f"✓ Saved: {filename} ({len(section_content)} chars)")

print()
print(f"✓ Saved {len(saved_chunks)} section chunks to {chunks_dir}")


Splitting markdown into 16 section chunks...
Output directory: /Users/lauren/repos/PAD2Skills/data/silver/pads_md_chunks

✓ Saved: P075941_0_strategic_context.md (31030 chars)
✓ Saved: P075941_1_project_development_objectives.md (2769 chars)
✓ Saved: P075941_2_project_description.md (17649 chars)
✓ Saved: P075941_3_implementation.md (8975 chars)
✓ Saved: P075941_4_key_risks_and_mitigation_measures.md (10570 chars)
✓ Saved: P075941_5_appraisal_summary.md (48636 chars)
✓ Saved: P075941_6_results_framework_and_monitoring.md (15964 chars)
✓ Saved: P075941_7_detailed_project_description.md (22009 chars)
✓ Saved: P075941_8_implementation_arrangements_regional_rusumo_falls_hydroelectric_project.md (101151 chars)
✓ Saved: P075941_9_operational_risk_assessment_framework_oraf.md (116773 chars)
✓ Saved: P075941_10_economic_and_financial_analysis_implementation_arrangements.md (189523 chars)
✓ Saved: P075941_11_power_supply_options_for_the_nile_equatorial_lakes_region_nel.md (10695 chars)
✓ Saved:

## 7. Verify Chunks Created

In [14]:
# List all created chunk files
chunk_files = sorted(chunks_dir.glob(f"{project_id}_*.md"))

print(f"Created {len(chunk_files)} chunk files:")
print("=" * 80)

for chunk_file in chunk_files:
    size_kb = chunk_file.stat().st_size / 1024
    print(f"  {chunk_file.name:60s} {size_kb:6.2f} KB")

print("=" * 80)
print(f"Total chunks: {len(chunk_files)}")


Created 16 chunk files:
  P075941_0_strategic_context.md                                30.30 KB
  P075941_10_economic_and_financial_analysis_implementation_arrangements.md 185.08 KB
  P075941_11_power_supply_options_for_the_nile_equatorial_lakes_region_nel.md  10.44 KB
  P075941_12_summary_of_the_power_sectors_in_burundi_rwanda_and_tanzania.md  32.54 KB
  P075941_13_implementation_support_team.md                      5.18 KB
  P075941_14_communication_strategy.md                          16.52 KB
  P075941_15_documents_in_the_project_file.md                    4.60 KB
  P075941_1_project_development_objectives.md                    2.70 KB
  P075941_2_project_description.md                              17.24 KB
  P075941_3_implementation.md                                    8.76 KB
  P075941_4_key_risks_and_mitigation_measures.md                10.32 KB
  P075941_5_appraisal_summary.md                                47.50 KB
  P075941_6_results_framework_and_monitoring.md            

## 8. Extract Abbreviations Section

In [15]:
# Reload markdown content
markdown_content = target_file.read_text(encoding="utf-8")

print(f"✓ Reloaded markdown content")
print(f"  Length: {len(markdown_content):,} characters")
print(f"  File: {target_file.name}")

✓ Reloaded markdown content
  Length: 736,923 characters
  File: P075941_1.md


### 8.01 Extract Text Up to First Table

In [20]:
import re

# Find the first occurrence of "table" (case insensitive)
match = re.search(r'\btable\b', markdown_content, re.IGNORECASE)

if match:
    # Extract text from beginning up to the first "table"
    abbreviations_text = markdown_content[:match.start()].strip()
    
    print(f"✓ Found first occurrence of 'table' at position {match.start()}")
    print(f"  Extracted {len(abbreviations_text)} characters")
    print(f"\nExtracted text preview (2000-3000 chars):")
    print("=" * 80)
    print(abbreviations_text[2000:3000])
else:
    print("✗ No occurrence of 'table' found in markdown content")
    abbreviations_text = ""

✓ Found first occurrence of 'table' at position 8472
  Extracted 8471 characters

Extracted text preview (2000-3000 chars):
 Supply  Company  Ltd.

DflD

Department  for  International Development, UK

DP

Development  Partner

DRC

Democratic  Republic of  Congo

E&amp;M

Electrical and  Mechanical

E&amp;S

Erosion  and Safety

EAPP

East  Africa Power  Pool

EARP

Electricity Access  Roll-out Program

EDPRS

Economic Development  and  Poverty Reduction  Strategy

EIRR

Economic Internal Rate  of Return

ENSAP

Eastern Nile  Subsidiary Action  Program

EPC

Engineering- Procurement- Construction

ESMP

Environment  and  Social Management Plan

ESMU

Environmental and  Social Management Unit

EWSA

Energy,  Water  and  Sanitation Authority, Rwanda

EWURA

Energy  and  Water  Utilities Regulatory Authority

FBU

Burundi  Franc

ABER

| FDS       | Full  Development  Scheme                                 |
|-----------|-----------------------------------------------------------|
| FIRR

### 8.02 Send to OpenAI API for Table Extraction

In [21]:
print("Sending abbreviations text to OpenAI API...")
print(f"Input length: {len(abbreviations_text)} chars")
print()

# Call custom GPT with prompt ID
response = client.responses.create(
    prompt={
        "id": "pmpt_69574fe1ff748195b16eca5fcc20d75202a70098695be261",
        "version": "2"
    },
    input=[
        {"role": "user", "content": abbreviations_text}
    ],
    reasoning={
        "summary": None
    },
    store=True,
    include=[
        "reasoning.encrypted_content",
        "web_search_call.action.sources"
    ]
)

# Extract the markdown table from the response
for item in response.output:
    if hasattr(item, 'content') and hasattr(item, 'role'):
        abbr_table = item.content[0].text
        break

print("✓ Response received")
print(f"  Response ID: {response.id}")
print(f"  Status: {response.status}")
print(f"  Table length: {len(abbr_table)} chars")
print(f"\nTable preview (first 500 chars):")
print("=" * 80)
print(abbr_table[:500])
print("...")

Sending abbreviations text to OpenAI API...
Input length: 8471 chars

✓ Response received
  Response ID: resp_039b882a1337aaae0069575232cae4819498db748bf065574e
  Status: completed
  Table length: 4506 chars

Table preview (first 500 chars):
Abbreviation | Definition
--- | ---
AfDB | Africa Development Bank
BoQ | Bill of Quantities
CAG | Controller and Auditor General
CAS | Country Assistance Strategy
CBWS | Comprehensive Basin-wide Study
COD | Commercial Operation Date
COMESA | Common Market for Eastern and Southern Africa
CSO | Civil Society Organization
DARESCO | District Electric Supply Company Ltd.
DflD | Department for International Development, UK
DP | Development Partner
DRC | Democratic Republic of Congo
E&M | Electrical an
...


### 8.03 Save Abbreviations Table

In [22]:
# Create output directory for abbreviations
abbr_output_dir = project_root / "data" / "silver" / "abbreviations_md"
abbr_output_dir.mkdir(parents=True, exist_ok=True)

# Save abbreviations table
abbr_output_file = abbr_output_dir / f"{project_id}_abbr.md"

with open(abbr_output_file, 'w', encoding='utf-8') as f:
    f.write(abbr_table)

print(f"✓ Saved abbreviations table to: {abbr_output_file}")
print(f"  File size: {abbr_output_file.stat().st_size / 1024:.2f} KB")

✓ Saved abbreviations table to: /Users/lauren/repos/PAD2Skills/data/silver/abbreviations_md/P075941_abbr.md
  File size: 4.40 KB
